In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import datetime

In [2]:
df_original = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)

In [3]:
df_features_categoricos = pd.read_csv('features_categoricos.csv', low_memory=False)

In [4]:
df_features_categoricos.head()

,person,modelo_mas_visitado,region_persona,pais,evento_predominante_mes_5
0,4886f805,Samsung Galaxy J7 Prime,Rio de Janeiro,Brazil,viewed product
1,ad93850f,iPhone 5s,Sao Paulo,Brazil,viewed product
2,0297fc1e,iPhone 6,Rio de Janeiro,Brazil,viewed product
3,2d681dd8,iPhone 7,Sao Paulo,Brazil,viewed product
4,cccea85e,Motorola Moto G4 Plus,Sao Paulo,Brazil,viewed product


In [5]:
df_features_categoricos.shape

(38829, 5)

Voy a marcar cual fue la semana del mes que mas registros tiene de cada persona

In [6]:
df_semana_persona = df_original[['person','timestamp','event']]

In [7]:
df_semana_persona.shape

(2341681, 3)

In [8]:
df_semana_persona['datetime'] = pd.to_datetime(df_semana_persona.loc[:,'timestamp'])

/home/amaherok/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
df_semana_persona['mes'] = df_semana_persona['datetime'].dt.month

/home/amaherok/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
df_semana_persona['dia'] = df_semana_persona['datetime'].dt.day

/home/amaherok/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
df_semana_persona.head()

,person,timestamp,event,datetime,mes,dia
0,4886f805,2018-05-18 00:11:59,viewed product,2018-05-18 00:11:59,5,18
1,ad93850f,2018-05-18 00:11:27,viewed product,2018-05-18 00:11:27,5,18
2,0297fc1e,2018-05-18 00:11:16,viewed product,2018-05-18 00:11:16,5,18
3,2d681dd8,2018-05-18 00:11:14,viewed product,2018-05-18 00:11:14,5,18
4,cccea85e,2018-05-18 00:11:09,viewed product,2018-05-18 00:11:09,5,18


In [12]:
df_semana_persona['semana_del_dia'] = (df_semana_persona['dia']-1) // 7 + 1

/home/amaherok/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
df_semana_persona = df_semana_persona.groupby(['person','mes','semana_del_dia']).size().reset_index()

In [14]:
df_semana_persona.groupby('person').get_group('00091926')

,person,mes,semana_del_dia,0
1,00091926,5,1,44
2,00091926,5,2,121
3,00091926,5,3,188
4,00091926,5,4,76
5,00091926,5,5,19


In [15]:
df_semana_persona['mes'].value_counts()

5    59589
4    11735
3     6802
2     2818
1     2003
Name: mes, dtype: int64

In [16]:
df_semana_persona = df_semana_persona.groupby(['person','semana_del_dia'])[0].sum()

In [17]:
df_semana_persona = df_semana_persona.reset_index()

In [18]:
df_semana_persona.sort_values(by=0,ascending=False).head()

,person,semana_del_dia,0
54222,c76b8417,1,1913
29302,6abd2bf1,4,1078
39108,8fb4929e,4,1073
48454,b1f4dbf6,4,1054
42689,9ccf882a,4,1045


In [19]:
df_semana_1 = df_semana_persona.groupby('semana_del_dia').get_group(1).rename(columns={0:'registros_semana_1'})[['person','registros_semana_1']]
df_semana_2 = df_semana_persona.groupby('semana_del_dia').get_group(2).rename(columns={0:'registros_semana_2'})[['person','registros_semana_2']]
df_semana_3 = df_semana_persona.groupby('semana_del_dia').get_group(3).rename(columns={0:'registros_semana_3'})[['person','registros_semana_3']]
df_semana_4 = df_semana_persona.groupby('semana_del_dia').get_group(4).rename(columns={0:'registros_semana_4'})[['person','registros_semana_4']]
df_semana_5 = df_semana_persona.groupby('semana_del_dia').get_group(5).rename(columns={0:'registros_semana_5'})[['person','registros_semana_5']]

In [20]:
df_semana_2

,person,registros_semana_2
2,00091926,121
21,0019e639,125
28,001ca5ee,14
31,001dfc31,9
40,00202cdb,17
43,0022965d,31
47,0027574e,68
51,002aea56,44
66,003476c0,32
72,0038a117,11


In [21]:
df_personas_totales = df_original.loc[:,['person']].drop_duplicates()

In [22]:
df_personas_totales = df_personas_totales.merge(df_semana_1, on='person', how='left')
df_personas_totales = df_personas_totales.merge(df_semana_2, on='person', how='left')
df_personas_totales = df_personas_totales.merge(df_semana_3, on='person', how='left')
df_personas_totales = df_personas_totales.merge(df_semana_4, on='person', how='left')
df_personas_totales = df_personas_totales.merge(df_semana_5, on='person', how='left')

In [23]:
df_personas_totales.head()

,person,registros_semana_1,registros_semana_2,registros_semana_3,registros_semana_4,registros_semana_5
0,4886f805,NaN,NaN,9.0,NaN,NaN
1,ad93850f,NaN,13.0,45.0,7.0,NaN
2,0297fc1e,67.0,99.0,151.0,200.0,50.0
3,2d681dd8,NaN,NaN,10.0,16.0,NaN
4,cccea85e,104.0,213.0,249.0,86.0,184.0


In [24]:
df_semana_con_mayor_visitas = df_semana_persona.sort_values(by=0,ascending=False)\
                                                .drop_duplicates(subset='person',keep='first')

In [25]:
df_semana_con_mayor_visitas = df_semana_con_mayor_visitas[['person','semana_del_dia']]\
                                                        .rename(columns={'semana_del_dia':'semana_mas_interactuante'})

In [26]:
def nombre_semana(x):
    nombres = {1:'First',2:'Second',3:'Third', 4:'Fourth', 5:'Fifth'}
    return nombres[x]


df_semana_con_mayor_visitas['semana_mas_interactuante'] = df_semana_con_mayor_visitas['semana_mas_interactuante']\
                                                            .apply(lambda x: nombre_semana(x))

In [27]:
df_personas_totales = df_personas_totales.merge(df_semana_con_mayor_visitas, on='person', how='left')

In [28]:
df_personas_totales['semana_mas_interactuante'].value_counts()

Fourth    13633
Third     12921
Fifth      7949
Second     2588
First      1738
Name: semana_mas_interactuante, dtype: int64

In [29]:
df_features_categoricos = df_features_categoricos.merge(df_personas_totales[['person','semana_mas_interactuante']],\
                                                        on='person', how='left')

In [30]:
df_features_categoricos.head(2)

,person,modelo_mas_visitado,region_persona,pais,evento_predominante_mes_5,semana_mas_interactuante
0,4886f805,Samsung Galaxy J7 Prime,Rio de Janeiro,Brazil,viewed product,Third
1,ad93850f,iPhone 5s,Sao Paulo,Brazil,viewed product,Third


Agrego al *features_numerico.csv* los items numericos

In [31]:
df_features_numericos = pd.read_csv('features_numericos.csv', low_memory=False)

In [32]:
df_features_numericos.head()

,person,Cantidad_visitas,Promedio_visitas_producto,horas_mirando_productos,visitas_mes1,visitas_mes2,visitas_mes3,visitas_mes4,visitas_mes5,promedio_ingreso_mensual
0,4886f805,4.0,5.043919,0.315278,0.0,0.0,0.0,0.0,9.0,1.8
1,ad93850f,17.0,9.904207,22.058056,0.0,0.0,0.0,0.0,65.0,13.0
2,0297fc1e,213.0,9.325721,191.342500,120.0,48.0,87.0,139.0,173.0,113.4
3,2d681dd8,6.0,8.860133,0.092222,0.0,0.0,0.0,0.0,26.0,5.2
4,cccea85e,252.0,9.100159,50.758611,0.0,0.0,0.0,0.0,836.0,167.2


In [33]:
df_personas_totales.head(2)

,person,registros_semana_1,registros_semana_2,registros_semana_3,registros_semana_4,registros_semana_5,semana_mas_interactuante
0,4886f805,NaN,NaN,9.0,NaN,NaN,Third
1,ad93850f,NaN,13.0,45.0,7.0,NaN,Third


In [34]:
df_features_numericos = df_features_numericos\
.merge(df_personas_totales[['person',
                            'registros_semana_1',\
                            'registros_semana_2',\
                            'registros_semana_3',\
                            'registros_semana_4',\
                            'registros_semana_5']],\
       on='person', how='left')

In [35]:
df_features_numericos.head()

,person,Cantidad_visitas,Promedio_visitas_producto,horas_mirando_productos,visitas_mes1,visitas_mes2,visitas_mes3,visitas_mes4,visitas_mes5,promedio_ingreso_mensual,registros_semana_1,registros_semana_2,registros_semana_3,registros_semana_4,registros_semana_5
0,4886f805,4.0,5.043919,0.315278,0.0,0.0,0.0,0.0,9.0,1.8,NaN,NaN,9.0,NaN,NaN
1,ad93850f,17.0,9.904207,22.058056,0.0,0.0,0.0,0.0,65.0,13.0,NaN,13.0,45.0,7.0,NaN
2,0297fc1e,213.0,9.325721,191.342500,120.0,48.0,87.0,139.0,173.0,113.4,67.0,99.0,151.0,200.0,50.0
3,2d681dd8,6.0,8.860133,0.092222,0.0,0.0,0.0,0.0,26.0,5.2,NaN,NaN,10.0,16.0,NaN
4,cccea85e,252.0,9.100159,50.758611,0.0,0.0,0.0,0.0,836.0,167.2,104.0,213.0,249.0,86.0,184.0


In [36]:
df_features_numericos.to_csv(path_or_buf = 'features_numericos.csv',index = False )

Categoria dependiendo de si ingreso via tablet o smartphone

In [37]:
df_tipo_de_ingreso = df_original.loc[:,['person','device_type','event']]

In [38]:
df_tipo_de_ingreso = df_tipo_de_ingreso.groupby(['person','device_type']).size().reset_index()

In [39]:
df_tipo_de_ingreso = df_tipo_de_ingreso.sort_values(by=0, ascending=False).drop_duplicates(subset='person', keep='first')

In [40]:
df_tipo_de_ingreso = df_tipo_de_ingreso.drop(columns=[0])

In [41]:
df_tipo_de_ingreso.shape

(38242, 2)

In [42]:
df_features_categoricos = df_features_categoricos.merge(df_tipo_de_ingreso, on='person', how='left')

In [43]:
df_features_categoricos.head()

,person,modelo_mas_visitado,region_persona,pais,evento_predominante_mes_5,semana_mas_interactuante,device_type
0,4886f805,Samsung Galaxy J7 Prime,Rio de Janeiro,Brazil,viewed product,Third,Smartphone
1,ad93850f,iPhone 5s,Sao Paulo,Brazil,viewed product,Third,Smartphone
2,0297fc1e,iPhone 6,Rio de Janeiro,Brazil,viewed product,Fourth,Smartphone
3,2d681dd8,iPhone 7,Sao Paulo,Brazil,viewed product,Fourth,Computer
4,cccea85e,Motorola Moto G4 Plus,Sao Paulo,Brazil,viewed product,Third,Computer


Tipo de channel de donde provienen

In [44]:
df_channel_person = df_original.loc[:,['person','channel','event']]

In [45]:
df_channel_person = df_channel_person.dropna()

In [46]:
df_channel_person = df_channel_person.groupby(['person','channel']).size().reset_index()

In [47]:
df_channel_person = df_channel_person.sort_values(by=0, ascending=False).drop_duplicates(subset='person', keep='first')

In [48]:
df_channel_person = df_channel_person.drop(columns=[0])

In [49]:
df_channel_person.columns = ['person','channel_frecuente']

In [50]:
df_features_categoricos = df_features_categoricos.merge(df_channel_person, on='person', how='left')

In [51]:
df_features_categoricos = df_features_categoricos.iloc[:,:8]

In [68]:
df_features_categoricos.head()

,person,region_persona,pais,evento_predominante_mes_5,semana_mas_interactuante,device_type,channel_frecuente,marca_mas_buscada
0,4886f805,Rio de Janeiro,Brazil,viewed product,Third,Smartphone,Organic,Samsung
1,ad93850f,Sao Paulo,Brazil,viewed product,Third,Smartphone,Paid,iPhone
2,0297fc1e,Rio de Janeiro,Brazil,viewed product,Fourth,Smartphone,Direct,iPhone
3,2d681dd8,Sao Paulo,Brazil,viewed product,Fourth,Computer,Organic,iPhone
4,cccea85e,Sao Paulo,Brazil,viewed product,Third,Computer,Organic,Motorola


In [53]:
df_features_categoricos.columns = ['person',\
                                   'modelo_mas_visitado',\
                                   'region_persona',\
                                   'pais',\
                                   'evento_predominante_mes_5',\
                                   'semana_mas_interactuante',\
                                   'device_type',\
                                   'channel_frecuente']

In [69]:
df_features_categoricos.head()

,person,region_persona,pais,evento_predominante_mes_5,semana_mas_interactuante,device_type,channel_frecuente,marca_mas_buscada
0,4886f805,Rio de Janeiro,Brazil,viewed product,Third,Smartphone,Organic,Samsung
1,ad93850f,Sao Paulo,Brazil,viewed product,Third,Smartphone,Paid,iPhone
2,0297fc1e,Rio de Janeiro,Brazil,viewed product,Fourth,Smartphone,Direct,iPhone
3,2d681dd8,Sao Paulo,Brazil,viewed product,Fourth,Computer,Organic,iPhone
4,cccea85e,Sao Paulo,Brazil,viewed product,Third,Computer,Organic,Motorola


In [55]:
df_features_categoricos.to_csv(path_or_buf='features_categoricos.csv', index=False)

# Rework features

In [56]:
def red_paises(x):
    if x not in ['Brazil','Argentina','United States','Canada']:
        return 'Another'
    return x

In [57]:
df_features_categoricos = pd.read_csv('features_categoricos.csv', low_memory=False)

In [58]:
df_features_categoricos.groupby('person')['region_persona'].value_counts()

person    region_persona    
0008ed71  0                     1
00091926  Rio Grande do Sul     1
00091a7a  Minas Gerais          1
000ba417  Sao Paulo             1
000c79fe  Minas Gerais          1
000e4d9e  Minas Gerais          1
000e619d  Sao Paulo             1
001001be  Sao Paulo             1
0010e89a  Sao Paulo             1
0016c4b5  Sao Paulo             1
001802e4  Rio de Janeiro        1
001804a2  0                     1
0019e639  Sao Paulo             1
001a2273  Unknown               1
001b0bf9  0                     1
001ca5ee  Sao Paulo             1
001dfc31  Sao Paulo             1
001e9aea  Unknown               1
0020152e  Rio Grande do Sul     1
00202cdb  Sao Paulo             1
0022965d  Sao Paulo             1
0027574e  Minas Gerais          1
00295d74  Unknown               1
002aea56  Mato Grosso           1
002b0188  Unknown               1
002e74b2  Unknown               1
002ed810  Santa Catarina        1
002fb981  Minas Gerais          1
00317e49  Parana   

In [59]:
df_features_categoricos.head()

,person,modelo_mas_visitado,region_persona,pais,evento_predominante_mes_5,semana_mas_interactuante,device_type,channel_frecuente
0,4886f805,Samsung Galaxy J7 Prime,Rio de Janeiro,Brazil,viewed product,Third,Smartphone,Organic
1,ad93850f,iPhone 5s,Sao Paulo,Brazil,viewed product,Third,Smartphone,Paid
2,0297fc1e,iPhone 6,Rio de Janeiro,Brazil,viewed product,Fourth,Smartphone,Direct
3,2d681dd8,iPhone 7,Sao Paulo,Brazil,viewed product,Fourth,Computer,Organic
4,cccea85e,Motorola Moto G4 Plus,Sao Paulo,Brazil,viewed product,Third,Computer,Organic


In [60]:
def red_modelos(x):
    
    if 'iPhone' in str(x):
        return 'iPhone'
    if 'Samsung' in str(x):
        return 'Samsung'
    if 'Motorola' in str(x):
        return 'Motorola'
    if 'LG' in str(x):
        return 'LG'
    return 'Other'

In [61]:
df_features_categoricos['marca_mas_buscada'] = df_features_categoricos['modelo_mas_visitado'].apply(lambda x: red_modelos(x))
df_features_categoricos['pais'] = df_features_categoricos['pais'].apply(lambda x: red_paises(x))

In [62]:
df_features_categoricos.head()

,person,modelo_mas_visitado,region_persona,pais,evento_predominante_mes_5,semana_mas_interactuante,device_type,channel_frecuente,marca_mas_buscada
0,4886f805,Samsung Galaxy J7 Prime,Rio de Janeiro,Brazil,viewed product,Third,Smartphone,Organic,Samsung
1,ad93850f,iPhone 5s,Sao Paulo,Brazil,viewed product,Third,Smartphone,Paid,iPhone
2,0297fc1e,iPhone 6,Rio de Janeiro,Brazil,viewed product,Fourth,Smartphone,Direct,iPhone
3,2d681dd8,iPhone 7,Sao Paulo,Brazil,viewed product,Fourth,Computer,Organic,iPhone
4,cccea85e,Motorola Moto G4 Plus,Sao Paulo,Brazil,viewed product,Third,Computer,Organic,Motorola


In [63]:
df_features_categoricos = df_features_categoricos.drop(columns= ['modelo_mas_visitado'])

In [64]:
df_features_categoricos['region_persona'].describe()

count         38829
unique          100
top       Sao Paulo
freq          10509
Name: region_persona, dtype: object

In [65]:
df_features_categoricos_red = df_features_categoricos.loc[:,['person', 'region_persona', 'pais', 'semana_mas_interactuante', 'device_type','channel_frecuente', 'marca_mas_buscada']]

In [66]:
df_features_categoricos_red.to_csv(path_or_buf = 'features_categoricos_reducidos.csv',index = False )

In [67]:
df_features_categoricos_red.head()

,person,region_persona,pais,semana_mas_interactuante,device_type,channel_frecuente,marca_mas_buscada
0,4886f805,Rio de Janeiro,Brazil,Third,Smartphone,Organic,Samsung
1,ad93850f,Sao Paulo,Brazil,Third,Smartphone,Paid,iPhone
2,0297fc1e,Rio de Janeiro,Brazil,Fourth,Smartphone,Direct,iPhone
3,2d681dd8,Sao Paulo,Brazil,Fourth,Computer,Organic,iPhone
4,cccea85e,Sao Paulo,Brazil,Third,Computer,Organic,Motorola
